In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

# reading the ratings data
ratings = pd.read_csv('ml-latest/ratings2.csv', sep=",")

# Making the dataset a little bit smaller due to lack of memory resources
# ratings = ratings.head(len(ratings)//100)
# print(ratings)

ratings_pivot = pd.pivot_table(ratings[['userId', 'movieId', 'rating']], 
                               values='rating', index='userId', columns='movieId' ).fillna(0)


# creating train and test sets
X_train, X_test = train_test_split(ratings_pivot, train_size=0.8)
# print(X_train)


In [18]:
# ratings.to_csv("ratings2.csv", index=False)

In [7]:
# Deciding how many nodes each layer should have - Depending on the dataset's size
movies_size = 15159 #58099
n_nodes_inpl = movies_size
n_nodes_hl1  = 256
n_nodes_outl = movies_size

# first hidden layer has 15159*256 weights and 256 biases
hidden_1_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_inpl+1, n_nodes_hl1]))}
output_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1+1, n_nodes_outl]))}

input_layer = tf.placeholder('float', [None, movies_size])

input_layer_const = tf.fill([tf.shape(input_layer)[0], 1], 1.0)
input_layer_concat = tf.concat([input_layer, input_layer_const], 1)

layer_1 = tf.nn.sigmoid(tf.matmul(input_layer_concat, hidden_1_layer_vals['weights']))

layer1_const = tf.fill( [tf.shape(layer_1)[0], 1], 1.0)
layer_concat =  tf.concat([layer_1, layer1_const], 1)

output_layer = tf.matmul(layer_concat, output_layer_vals['weights'])
output_true = tf.placeholder('float', [None, movies_size])
meansq = tf.reduce_mean(tf.square(output_layer - output_true))

learn_rate = 0.1   # learning rate
optimizer = tf.train.AdagradOptimizer(learn_rate).minimize(meansq)

# initializing variables and starting the session
init = tf.global_variables_initializer()
sess = tf.Session()
writer = tf.summary.FileWriter('./graphs', tf.get_default_graph())
writer.close()
sess.run(init)    


## Autoencoder with one hidden layer

![autoencoder with one layer](Images/autoencoders-1layer.png)

In [8]:
batch_size = 100
hm_epochs = 10000
tot_users = X_train.shape[0]
# print(tot_users)

for epoch in range(hm_epochs):
    epoch_loss = 0    # initializing error as 0
    for i in range(int(tot_users/batch_size)):
        # print(epoch_x)
        epoch_x = X_train[ i*batch_size : (i+1)*batch_size ]
        _, c = sess.run([optimizer, meansq],feed_dict={input_layer: epoch_x, output_true: epoch_x})
        epoch_loss += c
        
    output_train = sess.run(output_layer, feed_dict={input_layer:X_train})
    output_test = sess.run(output_layer, feed_dict={input_layer:X_test})
    print('MSE train', MSE(output_train, X_train),'MSE test', MSE(output_test, X_test))      
    print('Epoch', epoch, '/', hm_epochs, 'loss:',epoch_loss)



MSE train 103.61305168161442 MSE test 104.39914391376686
Epoch 0 / 10000 loss: 2405.342124938965
MSE train 88.75391705831747 MSE test 89.58262378900004
Epoch 1 / 10000 loss: 2027.1185836791992
MSE train 77.71646725737037 MSE test 78.42167132517308
Epoch 2 / 10000 loss: 1756.1263122558594
MSE train 69.20817850482163 MSE test 69.84069429708786
Epoch 3 / 10000 loss: 1550.3154525756836
MSE train 62.39136858247147 MSE test 62.978598688487715
Epoch 4 / 10000 loss: 1389.608169555664
MSE train 56.89096179580607 MSE test 57.536959658152426
Epoch 5 / 10000 loss: 1259.2187118530273
MSE train 52.49309155083232 MSE test 53.075452227451635
Epoch 6 / 10000 loss: 1155.0602836608887
MSE train 48.78740183551802 MSE test 49.393628651979654
Epoch 7 / 10000 loss: 1070.0268630981445
MSE train 45.614263571740516 MSE test 46.28982381901562
Epoch 8 / 10000 loss: 997.1967010498047
MSE train 42.9860053302266 MSE test 43.695583102031826
Epoch 9 / 10000 loss: 935.8436622619629
MSE train 40.70990814533569 MSE test 

MSE train 14.347830964585178 MSE test 16.05289847703229
Epoch 85 / 10000 loss: 301.51487922668457
MSE train 14.265031369241884 MSE test 15.977345792768855
Epoch 86 / 10000 loss: 299.721887588501
MSE train 14.182690514347005 MSE test 15.903104185639183
Epoch 87 / 10000 loss: 297.9803466796875
MSE train 14.103194935855447 MSE test 15.830429699153134
Epoch 88 / 10000 loss: 296.2491798400879
MSE train 14.02569952997984 MSE test 15.759507713350379
Epoch 89 / 10000 loss: 294.5789852142334
MSE train 13.949768790126791 MSE test 15.68988547338055
Epoch 90 / 10000 loss: 292.9507713317871
MSE train 13.876212159959056 MSE test 15.621266681919566
Epoch 91 / 10000 loss: 291.35611724853516
MSE train 13.8041779030753 MSE test 15.55430867119352
Epoch 92 / 10000 loss: 289.81330013275146
MSE train 13.73326700101128 MSE test 15.488722012825066
Epoch 93 / 10000 loss: 288.29965019226074
MSE train 13.663687713365151 MSE test 15.424808193793277
Epoch 94 / 10000 loss: 286.8103675842285
MSE train 13.59413209552

MSE train 10.303424151204766 MSE test 12.422860461485197
Epoch 168 / 10000 loss: 215.16175270080566
MSE train 10.274456829495092 MSE test 12.39801155732604
Epoch 169 / 10000 loss: 214.54729557037354
MSE train 10.246223828653138 MSE test 12.373480334156046
Epoch 170 / 10000 loss: 213.93572807312012
MSE train 10.218542548247218 MSE test 12.349216087012598
Epoch 171 / 10000 loss: 213.3395791053772
MSE train 10.190970989421691 MSE test 12.32499634076911
Epoch 172 / 10000 loss: 212.75478315353394
MSE train 10.16350022989562 MSE test 12.300967821043695
Epoch 173 / 10000 loss: 212.1729712486267
MSE train 10.13607721179628 MSE test 12.27710465097494
Epoch 174 / 10000 loss: 211.59249687194824
MSE train 10.109396232875913 MSE test 12.253633783280327
Epoch 175 / 10000 loss: 211.0140085220337
MSE train 10.082666469501996 MSE test 12.230290411063944
Epoch 176 / 10000 loss: 210.45083618164062
MSE train 10.055843533837466 MSE test 12.206974141248162
Epoch 177 / 10000 loss: 209.8862180709839
MSE train

MSE train 8.452699313274556 MSE test 10.824940638733462
Epoch 252 / 10000 loss: 175.84223318099976
MSE train 8.436372634872994 MSE test 10.81083649781613
Epoch 253 / 10000 loss: 175.49571990966797
MSE train 8.420148189403914 MSE test 10.796839345236375
Epoch 254 / 10000 loss: 175.151602268219
MSE train 8.40419677311923 MSE test 10.782988702903669
Epoch 255 / 10000 loss: 174.80966186523438
MSE train 8.388514320470172 MSE test 10.769282343771964
Epoch 256 / 10000 loss: 174.47372102737427
MSE train 8.372998602224776 MSE test 10.755713517598004
Epoch 257 / 10000 loss: 174.14361429214478
MSE train 8.357333725256398 MSE test 10.742268414522444
Epoch 258 / 10000 loss: 173.81700468063354
MSE train 8.341625872558163 MSE test 10.728921676089962
Epoch 259 / 10000 loss: 173.48702907562256
MSE train 8.325869500337639 MSE test 10.715684307175955
Epoch 260 / 10000 loss: 173.1560254096985
MSE train 8.310636010627801 MSE test 10.702628932210851
Epoch 261 / 10000 loss: 172.82423877716064
MSE train 8.295

MSE train 7.353162112660694 MSE test 9.922647157950166
Epoch 336 / 10000 loss: 152.54390287399292
MSE train 7.342384302224616 MSE test 9.913915504700702
Epoch 337 / 10000 loss: 152.312518119812
MSE train 7.331777237154438 MSE test 9.905204854162468
Epoch 338 / 10000 loss: 152.08468532562256
MSE train 7.321285502812657 MSE test 9.896541614654893
Epoch 339 / 10000 loss: 151.86043071746826
MSE train 7.311001307662432 MSE test 9.888011635682075
Epoch 340 / 10000 loss: 151.63869047164917
MSE train 7.3009053511422 MSE test 9.87959907702634
Epoch 341 / 10000 loss: 151.42144918441772
MSE train 7.290843924803988 MSE test 9.871268086278782
Epoch 342 / 10000 loss: 151.2081823348999
MSE train 7.280945297441915 MSE test 9.863023473627132
Epoch 343 / 10000 loss: 150.99561500549316
MSE train 7.27107332675544 MSE test 9.854872343758599
Epoch 344 / 10000 loss: 150.7865228652954
MSE train 7.261175860812706 MSE test 9.846767132262546
Epoch 345 / 10000 loss: 150.57784366607666
MSE train 7.251319999952886 

...

MSE train 2.699423227977067 MSE test 6.171356951950888
Epoch 3940 / 10000 loss: 54.00324070453644
MSE train 2.699212121372652 MSE test 6.171140404533751
Epoch 3941 / 10000 loss: 53.99877107143402
MSE train 2.699002286520496 MSE test 6.1709236840393595
Epoch 3942 / 10000 loss: 53.99434185028076
MSE train 2.6987930904848065 MSE test 6.170706607977018
Epoch 3943 / 10000 loss: 53.98994052410126
MSE train 2.698583943207626 MSE test 6.170489044798338
Epoch 3944 / 10000 loss: 53.9855535030365
MSE train 2.698374364132602 MSE test 6.170270840785883
Epoch 3945 / 10000 loss: 53.98116672039032
MSE train 2.6981638896141606 MSE test 6.170051898766563
Epoch 3946 / 10000 loss: 53.976768255233765
MSE train 2.6979520599007722 MSE test 6.169832010289316
Epoch 3947 / 10000 loss: 53.972352504730225
MSE train 2.6977383479532926 MSE test 6.1696110680050165
Epoch 3948 / 10000 loss: 53.967905163764954
MSE train 2.6975221508600886 MSE test 6.169388968900656
Epoch 3949 / 10000 loss: 53.96341872215271
MSE train 2

MSE train 2.6776231631886165 MSE test 6.151110631206372
Epoch 4024 / 10000 loss: 53.54466652870178
MSE train 2.6773373454770084 MSE test 6.1508640027698585
Epoch 4025 / 10000 loss: 53.538713693618774
MSE train 2.677049404811288 MSE test 6.150618205890602
Epoch 4026 / 10000 loss: 53.53269624710083
MSE train 2.6767612574246615 MSE test 6.150373640568064
Epoch 4027 / 10000 loss: 53.526636838912964
MSE train 2.676474493128886 MSE test 6.150130745763739
Epoch 4028 / 10000 loss: 53.52057731151581
MSE train 2.676188862580227 MSE test 6.149889958810136
Epoch 4029 / 10000 loss: 53.514556527137756
MSE train 2.6759022264422274 MSE test 6.149651705167704
Epoch 4030 / 10000 loss: 53.508567333221436
MSE train 2.675613134891249 MSE test 6.149415805541817
Epoch 4031 / 10000 loss: 53.502564907073975
MSE train 2.6753244740641033 MSE test 6.149182321868959
Epoch 4032 / 10000 loss: 53.49651741981506
MSE train 2.675042389124623 MSE test 6.148950936508067
Epoch 4033 / 10000 loss: 53.49048364162445
MSE train

MSE train 2.658742911166783 MSE test 6.132776566778491
Epoch 4107 / 10000 loss: 53.14889883995056
MSE train 2.6584749395303398 MSE test 6.132547625591384
Epoch 4108 / 10000 loss: 53.142826199531555
MSE train 2.658219926274915 MSE test 6.132321336902877
Epoch 4109 / 10000 loss: 53.137176752090454
MSE train 2.6579710619596044 MSE test 6.132096929066138
Epoch 4110 / 10000 loss: 53.13180720806122
MSE train 2.6577249169280517 MSE test 6.131873821393743
Epoch 4111 / 10000 loss: 53.12657606601715
MSE train 2.6574797612640064 MSE test 6.131651760190943
Epoch 4112 / 10000 loss: 53.121399998664856
MSE train 2.6572345740929015 MSE test 6.131430476251774
Epoch 4113 / 10000 loss: 53.11625158786774
MSE train 2.6569885703883465 MSE test 6.131209879123665
Epoch 4114 / 10000 loss: 53.111101269721985
MSE train 2.6567409760131366 MSE test 6.130989877734251
Epoch 4115 / 10000 loss: 53.105934858322144
MSE train 2.6564908267652054 MSE test 6.130770471537617
Epoch 4116 / 10000 loss: 53.10073280334473
MSE tra

MSE train 2.6385824682245502 MSE test 6.114923257740101
Epoch 4190 / 10000 loss: 52.72653114795685
MSE train 2.6383088327821023 MSE test 6.114709031222573
Epoch 4191 / 10000 loss: 52.720935583114624
MSE train 2.6380268086078975 MSE test 6.114495034631783
Epoch 4192 / 10000 loss: 52.715195536613464
MSE train 2.6377377658631893 MSE test 6.1142812944078
Epoch 4193 / 10000 loss: 52.70927679538727
MSE train 2.637447649767771 MSE test 6.114068217438216
Epoch 4194 / 10000 loss: 52.703205823898315
MSE train 2.637163724674694 MSE test 6.11385628022909
Epoch 4195 / 10000 loss: 52.69711422920227
MSE train 2.6368893753592926 MSE test 6.113645759833497
Epoch 4196 / 10000 loss: 52.69115960597992
MSE train 2.636624168883225 MSE test 6.113436447943559
Epoch 4197 / 10000 loss: 52.68541145324707
MSE train 2.6363664316263273 MSE test 6.113228155419297
Epoch 4198 / 10000 loss: 52.679866313934326
MSE train 2.636114236845357 MSE test 6.113020426439531
Epoch 4199 / 10000 loss: 52.674485206604004
MSE train 2.

KeyboardInterrupt: 

In [ ]:
# pick a user
sample_user = X_test.iloc[99,:]
# get the predicted ratings
sample_user_pred = sess.run(output_layer, feed_dict={input_layer:[sample_user]})

In [25]:
import pickle
v1 = sess.run(hidden_1_layer_vals)
f = open("weights.pkl","wb")
pickle.dump(v1,f)
f.close()

v2 = sess.run(output_layer_vals)
f = open("weights-output.pkl","wb")
pickle.dump(v2,f)
f.close()
